# Setup and inspect the model architectures

Model 1 uses pulse oximetry (PPG and SpO2) as inputs. Model 2 uses PPG, SpO2, and nasal pressure as inputs. Model 3 uses SpO2, nasal pressure, oronasal thermocouple, EEG, and RIPsum as inputs.

The sampling frequency for all input signals is 32 Hz.

Outputs for all models are the hypnogram (30-s segments; wake/N1/N2/N3/REM), and respiratory event classification (1-s segments; no event/hypopnea/apnea).

## Model 1

In [1]:
%pip install -e ..

Obtaining file:///home/richardson/code/psg-simultscoring-models
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: psg-simultscoring-models
    Found existing installation: psg-simultscoring-models 0.0.1.dev0
    Uninstalling psg-simultscoring-models-0.0.1.dev0:
      Successfully uninstalled psg-simultscoring-models-0.0.1.dev0
  Running setup.py develop for psg-simultscoring-models
Note: you may need to restart the kernel to use updated packages.


In [2]:
import psg_simultscoring_models.utime as utime
import tensorflow as tf

2022-08-10 13:59:13.982632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 13:59:13.982773: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
block_args_list = utime.DEFAULT_BLOCK_ARGS
for block_args in block_args_list:
    print(block_args)

BlockArgs(kernel_size=5, input_filters=None, output_filters=32, pool_size=8, dilation=2, strides=1, expand_ratio=6, se_ratio=0.25, id_skip=True, dec_skip=False)
BlockArgs(kernel_size=5, input_filters=None, output_filters=48, pool_size=6, dilation=2, strides=1, expand_ratio=6, se_ratio=0.25, id_skip=True, dec_skip=False)
BlockArgs(kernel_size=5, input_filters=None, output_filters=64, pool_size=4, dilation=2, strides=1, expand_ratio=6, se_ratio=0.25, id_skip=True, dec_skip=False)
BlockArgs(kernel_size=3, input_filters=None, output_filters=96, pool_size=2, dilation=1, strides=1, expand_ratio=6, se_ratio=0.25, id_skip=True, dec_skip=False)
BlockArgs(kernel_size=3, input_filters=None, output_filters=128, pool_size=2, dilation=1, strides=1, expand_ratio=6, se_ratio=0.1, id_skip=True, dec_skip=False)
BlockArgs(kernel_size=3, input_filters=None, output_filters=256, pool_size=1, dilation=1, strides=1, expand_ratio=6, se_ratio=0.1, id_skip=True, dec_skip=False)


In [4]:
fs = 32
output_args = [
    utime.OutputArgs(
        output_name='hypnogram',
        n_classes=5,
        samples_per_segment=fs*30,  # 30-s segments for sleep staging
        dense_ksize=1,  # Pointwise convolution in segment classifier
        segment_ksize=1
    ),
    utime.OutputArgs(
        output_name='obstruction',
        n_classes=3,
        samples_per_segment=fs*1,  # 1-s segments for respiratory event detection
        dense_ksize=1,  # Pointwise convolution in segment classifier
        segment_ksize=1
    )
]

In [5]:
aspp_depth = 512
m1_input_names = ['ppg', 'spo2']

In [6]:
model_1 = utime.UTimeF(
    (None, 1),
    block_args_list=block_args_list,
    enc_cls=utime.Encoder,
    dec_cls=utime.Decoder,
    conv_type='conv',
    aspp_depth=aspp_depth,
    input_names=m1_input_names,
    output_args=output_args
)

2022-08-10 13:59:43.134813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-10 13:59:43.135377: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 13:59:43.135924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5CG91645FF): /proc/driver/nvidia/version does not exist
2022-08-10 13:59:43.150540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model_1.summary()

Model: "u-time"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ppg (InputLayer)               [(None, None, 1)]    0           []                               
                                                                                                  
 spo2 (InputLayer)              [(None, None, 1)]    0           []                               
                                                                                                  
 concatenated (Concatenate)     (None, None, 2)      0           ['ppg[0][0]',                    
                                                                  'spo2[0][0]']                   
                                                                                                  
 encoder (Encoder)              [(None, None, 256),  517393      ['concatenated[0][0]']      

## Model 2

In [8]:
m2_input_names = ['ppg', 'spo2', 'naspres']

model_2 = utime.UTimeF(
    (None, 1),
    block_args_list=block_args_list,
    enc_cls=utime.Encoder,
    dec_cls=utime.Decoder,
    conv_type='conv',
    aspp_depth=aspp_depth,
    input_names=m2_input_names,
    output_args=output_args
)

In [9]:
model_2.summary()

Model: "u-time"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ppg (InputLayer)               [(None, None, 1)]    0           []                               
                                                                                                  
 spo2 (InputLayer)              [(None, None, 1)]    0           []                               
                                                                                                  
 naspres (InputLayer)           [(None, None, 1)]    0           []                               
                                                                                                  
 concatenated (Concatenate)     (None, None, 3)      0           ['ppg[0][0]',                    
                                                                  'spo2[0][0]',              

## Model 3

In [10]:
m3_input_names = ['spo2', 'naspres', 'thermocouple', 'ripsum', 'c4m1']

model_3 = utime.UTimeF(
    (None, 1),
    block_args_list=block_args_list,
    enc_cls=utime.Encoder,
    dec_cls=utime.Decoder,
    conv_type='conv',
    aspp_depth=aspp_depth,
    input_names=m3_input_names,
    output_args=output_args
)

In [11]:
model_3.summary()

Model: "u-time"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 spo2 (InputLayer)              [(None, None, 1)]    0           []                               
                                                                                                  
 naspres (InputLayer)           [(None, None, 1)]    0           []                               
                                                                                                  
 thermocouple (InputLayer)      [(None, None, 1)]    0           []                               
                                                                                                  
 ripsum (InputLayer)            [(None, None, 1)]    0           []                               
                                                                                             